<a href="https://colab.research.google.com/github/Fra1977/TF_FederatedLearning_Workshop/blob/dev/Quirkshop_2020_TFF_Intro_Tutorial_User_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The TensorFlow Authors.

## Before we start

Before we start, please run the following to make sure that your environment is
correctly setup. If you don't see a greeting, please refer to the
[Installation](../install.md) guide for instructions. 

In [2]:
#@title Upgrade tensorflow_federated and load TensorBoard
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow_federated
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

%load_ext tensorboard

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

     |████████████████████████████████| 481kB 7.1MB/s 
     |████████████████████████████████| 1.1MB 32.5MB/s 
     |████████████████████████████████| 174kB 41.2MB/s 
     |████████████████████████████████| 296kB 41.8MB/s 
     |████████████████████████████████| 122kB 44.2MB/s 
     |████████████████████████████████| 3.0MB 25.6MB/s 


In [3]:
import collections
from matplotlib import pyplot as plt
from IPython.display import display, HTML, IFrame

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from skimage import io

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

np.random.seed(0)

def greetings():
  display(HTML('<b><font size="6" color="#ff00f4">Greetings, virtual tutorial participants!</font></b>'))
  return True
l = tff.federated_computation(greetings)()

# Federated Learning for Image Classification

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/federated/blob/v0.14.0/docs/tutorials/federated_learning_for_image_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/federated/blob/v0.14.0/docs/tutorials/federated_learning_for_image_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

Let's experiment with federated learning in simulation. In this tutorial, we use the classic MNIST training example to introduce the
Federated Learning (FL) API layer of TFF, `tff.learning` - a set of
higher-level interfaces that can be used to perform common types of federated
learning tasks, such as federated training, against user-supplied models
implemented in TensorFlow.

<center><img src=https://storage.googleapis.com/tfds-data/visualization/mnist-3.0.1.png width="400">


# Tutorial Outline
We'll be training a model to perform image classification using the classic MNIST dataset, with the neural net learning to classify digit from image. In this case, we'll be simulating federated learning, with the training data distributed on different devices.

<p><b>Sections</b></p>


1.   Load TFF Libraries.
2.   Explore/preprocess federated EMNIST dataset.
3.   Create a model.
4.   Set up federated averaging process for training.
5.   Analyze training metrics.
6.   Set up federated evaluation computation.
7.   Analyze evaluation metrics.



## Preparing the input data

Let's start with the data. Federated learning requires a federated data set,
i.e., a collection of data from multiple users. Federated data is typically
non-[i.i.d.](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables),
which poses a unique set of challenges. Users typically have different distributions of data depending on usage patterns.

In order to facilitate experimentation, we seeded the TFF repository with a few
datasets.

Here's how we can load our sample dataset.

In [ ]:
# Your code here for loading the emnist dataset from the TFF repository.


The data sets returned by `load_data()` are instances of
`tff.simulation.ClientData`, an interface that allows you to enumerate the set
of users, to construct a `tf.data.Dataset` that represents the data of a
particular user, and to query the structure of individual elements.

Let's explore the dataset.

In [ ]:
len(emnist_train.client_ids)

In [ ]:
# Let's look at the shape of our data
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])

example_dataset.element_spec

In [ ]:
# Let's select an example dataset from one of our simulated clients
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])

# Your code to get an example element from one client:


In [ ]:
plt.imshow(example_element['pixels'].numpy(), cmap='gray', aspect='equal')
plt.grid(False)
_ = plt.show()

**Exploring non-iid data**

Use this time to ask questions on the dory if stuck / get caught up.
This section is separate from the rest of the tutorial.

In [ ]:
## Example MNIST digits for one client
f = plt.figure(figsize=(20,4))
j = 0

for e in example_dataset.take(40):
  plt.subplot(4, 10, j+1)
  plt.imshow(e['pixels'].numpy(), cmap='gray', aspect='equal')
  plt.axis('off')
  j += 1

In [ ]:
# Number of examples per layer for a sample of clients
f = plt.figure(figsize=(12,7))
f.suptitle("Label Counts for a Sample of Clients")
for i in range(6):
  ds = emnist_train.create_tf_dataset_for_client(emnist_train.client_ids[i])
  k = collections.defaultdict(list)
  for e in ds:
    k[e['label'].numpy()].append(e['label'].numpy())
  plt.subplot(2, 3, i+1)
  plt.title("Client {}".format(i))
  for j in range(10):
    plt.hist(k[j], density=False, bins=[0,1,2,3,4,5,6,7,8,9,10])


In [ ]:
# Let's play around with the emnist_train dataset.
# Let's explore the non-iid charateristic of the example data.

for i in range(5):
  ds = emnist_train.create_tf_dataset_for_client(emnist_train.client_ids[i])
  k = collections.defaultdict(list)
  for e in ds:
    k[e['label'].numpy()].append(e['pixels'].numpy())
  f = plt.figure(i, figsize=(12,5))
  f.suptitle("Client #{}'s Mean Image Per Label".format(i))
  for j in range(10):
    mn_img = np.mean(k[j],0)
    plt.subplot(2, 5, j+1)
    plt.imshow(mn_img.reshape((28,28)))#,cmap='gray') 
    plt.axis('off')

# Each client has different mean images -- each client will be nudging the model
# in their own directions.

### Preprocessing the data

Since the data is already a `tf.data.Dataset`,  preprocessing can be accomplished using Dataset transformations. [See here](https://www.tensorflow.org/guide/data) for more detail on these transformations.

In [ ]:
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER=10

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1, 784]),
        y=tf.reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

Let's verify this worked.

In [ ]:
preprocessed_example_dataset = preprocess(example_dataset)

sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_dataset)))

sample_batch

Here's a simple helper function that will construct a list of datasets from the
given set of users as an input to a round of training or evaluation.

In [ ]:
def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
  ]

Now, how do we choose clients?

In [ ]:
sample_clients = emnist_train.client_ids[0:NUM_CLIENTS]

# Your code to get the federated dataset here for the sampled clients:


print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))

## Creating a model with Keras

If you are using Keras, you likely already have code that constructs a Keras
model. Here's an example of a simple model that will suffice for our needs.

In [ ]:
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])

**Centralized training with Keras**

Use this time to ask questions on the dory if stuck / get caught up.
This section is separate from the rest of the tutorial.

In [ ]:
## Centralized training with keras ---------------------------------------------

# This is separate from the TFF tutorial, and demonstrates how to train a
# Keras model in a centralized fashion (contrasting training in a federated env)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 784).astype("float32") / 255

y_train = y_train.astype("float32")

mod = create_keras_model()
mod.compile(
    optimizer=tf.keras.optimizers.RMSprop(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
h = mod.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2
)

# ------------------------------------------------------------------------------



In order to use any model with TFF, it needs to be wrapped in an instance of the
`tff.learning.Model` interface.

More keras metrics you can add are [found here](https://www.tensorflow.org/api_docs/python/tf/keras/metrics).

In [ ]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## Training the model on federated data

Now that we have a model wrapped as `tff.learning.Model` for use with TFF, we
can let TFF construct a Federated Averaging algorithm by invoking the helper
function `tff.learning.build_federated_averaging_process`, as follows.

In [ ]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    # Add server optimizer here!
    

What just happened? TFF has constructed a pair of *federated computations* and
packaged them into a `tff.templates.IterativeProcess` in which these computations
are available as a pair of properties `initialize` and `next`.

An iterative process will usually be driven by a control loop like:
```
def initialize():
  ...

def next(state):
  ...

iterative_process = IterativeProcess(initialize, next)
state = iterative_process.initialize()
for round in range(num_rounds):
  state = iterative_process.next(state)
```



Let's invoke the `initialize` computation to construct the server state.

In [ ]:
state = iterative_process.initialize()

The second of the pair of federated computations, `next`, represents a single
round of Federated Averaging, which consists of pushing the server state
(including the model parameters) to the clients, on-device training on their
local data, collecting and averaging model updates, and producing a new updated
model at the server.

Let's run a single round of training and visualize the results. We can use the
federated data we've already generated above for a sample of users.

In [ ]:
# Run one single round of training.
state, metrics = iterative_process.next(state, federated_train_data)
print('round  1, metrics={}'.format(metrics['train']))

Let's run a few more rounds. As noted earlier, typically at this point you would
pick a subset of your simulation data from a new randomly selected sample of
users for each round in order to simulate a realistic deployment in which users
continuously come and go, but in this interactive notebook, for the sake of
demonstration we'll just reuse the same users, so that the system converges
quickly.

In [ ]:
NUM_ROUNDS = 11
for round_num in range(2, NUM_ROUNDS):
  state, metrics = iterative_process.next(state, federated_train_data)
  print('round {:2d}, metrics={}'.format(round_num, metrics['train']))

Training loss is decreasing after each round of federated training, indicating
the model is converging. There are some important caveats with these training
metrics, however, see the section on *Evaluation* later in this tutorial.

##Displaying model metrics in TensorBoard
Next, let's visualize the metrics from these federated computations using Tensorboard.

Let's start by creating the directory and the corresponding summary writer to write the metrics to.




In [ ]:
#@test {"skip": true}
import os
import shutil

logdir = "/tmp/logs/scalars/training/"
if os.path.exists(logdir):
  shutil.rmtree(logdir)

# Your code to create a summary writer:

state = iterative_process.initialize()

Plot the relevant scalar metrics with the same summary writer.

In [ ]:
#@test {"skip": true}
with summary_writer.as_default():
  for round_num in range(1, NUM_ROUNDS):
    state, metrics = iterative_process.next(state, federated_train_data)
    for name, value in metrics['train'].items():
      tf.summary.scalar(name, value, step=round_num)

Start TensorBoard with the root log directory specified above. It can take a few seconds for the data to load.

In [ ]:
#@test {"skip": true}
%tensorboard --logdir /tmp/logs/scalars/ --port=0

In order to view evaluation metrics the same way, you can create a separate eval folder, like "logs/scalars/eval", to write to TensorBoard.

## Evaluation
To perform evaluation on federated data, you can construct another *federated
computation* designed for just this purpose, using the
`tff.learning.build_federated_evaluation` function, and passing in your model
constructor as an argument.

In [ ]:
# Construct federated evaluation computation here:


Now, let's compile a test sample of federated data and rerun evaluation on the
test data. The data will come from a different sample of users, but from a
distinct held-out data set.

In [ ]:
import random
shuffled_ids = emnist_test.client_ids.copy()
random.shuffle(shuffled_ids)
sample_clients = shuffled_ids[0:NUM_CLIENTS]

federated_test_data = make_federated_data(emnist_test, sample_clients)

len(federated_test_data), federated_test_data[0]

In [ ]:
# Run evaluation on the test data here, using the federated model produced from 
# training:


In [ ]:
str(test_metrics)

This concludes the tutorial. We encourage you to play with the
parameters (e.g., batch sizes, number of users, epochs, learning rates, etc.), to modify the code above to simulate training on random samples of users in
each round, and to explore the other tutorials we've developed.

# Federated Learning for Text Generation

This tutorial builds on the concepts in the [Federated Learning for Image Classification](federated_learning_for_image_classification.ipynb) tutorial, and demonstrates several other useful approaches for federated learning.

In particular, we load a previously trained Keras model, and refine it using  federated training on a (simulated) decentralized dataset. This is practically important for several reasons. The ability to use serialized models makes it easy to mix federated learning with other ML approaches. Further, this allows use of an increasing range of pre-trained models --- for example, training language models from scratch is rarely necessary, as numerous pre-trained models are now widely available (see, e.g., [TF Hub](https://www.tensorflow.org/hub)). Instead, it makes more sense to start from a pre-trained model, and refine it using Federated Learning, adapting to the particular characteristics of the decentralized data for a particular application.

We will also demonstrate creating your own TFF dataset from a CSV file, so that you can begin using TFF on your own datasets.

For this tutorial, we start with a RNN that generates ASCII characters, and refine it via federated learning. We also show how the final weights can be fed back to the original Keras model, allowing easy evaluation and text generation using standard tools.

<p><b>Sections</b></p>

1.   Load and test an existing text generation model.
2.   Create a TFF dataset from a CSV file of Shakespeare data.
3.   Preprocess the data and test the model on the new data.
4.   Simulate cross-silo federated learning with centralized evaluation.
5.   Simulate cross-device federated learning with centralized evaluation.

## Load a pre-trained model

We load a model that was pre-trained following the TensorFlow tutorial
[Text generation using a RNN with eager execution](https://www.tensorflow.org/tutorials/sequences/text_generation). However,
rather than training on [The Complete Works of Shakespeare](http://www.gutenberg.org/files/100/100-0.txt), we pre-trained the model on the text from the Charles Dickens'
    [A Tale of Two Cities](http://www.ibiblio.org/pub/docs/books/gutenberg/9/98/98.txt)
    and
    [A Christmas Carol](http://www.ibiblio.org/pub/docs/books/gutenberg/4/46/46.txt).
 
Other than expanding the vocabulary, we didn't modify the original tutorial, so this initial model isn't state-of-the-art, but it produces reasonable predictions and is sufficient for our tutorial purposes. The final model was saved with `tf.keras.models.save_model(include_optimizer=False)`.
   
 We will use federated learning to fine-tune this model for Shakespeare in this tutorial, using a federated version of the data provided by TFF.



In [ ]:
import os

def load_model(batch_size):
  urls = {
      1: 'https://storage.googleapis.com/tff-models-public/dickens_rnn.batch1.kerasmodel',
      8: 'https://storage.googleapis.com/tff-models-public/dickens_rnn.batch8.kerasmodel'}
  assert batch_size in urls, 'batch_size must be in ' + str(urls.keys())
  url = urls[batch_size]
  local_file = tf.keras.utils.get_file(os.path.basename(url), origin=url)  
  return tf.keras.models.load_model(local_file, compile=False)

### Test the loaded model
Let's create some vocab lookup tables in order to generate some text.

In [ ]:
# A fixed vocabulary of ASCII chars that occur in the works of Shakespeare and Dickens:
vocab = list('dhlptx@DHLPTX $(,048cgkoswCGKOSW[_#\'/37;?bfjnrvzBFJNRVZ"&*.26:\naeimquyAEIMQUY]!%)-159\r')

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [ ]:
def generate_text(model, start_string):
  # From https://www.tensorflow.org/tutorials/sequences/text_generation
  num_generate = 200
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(
        predictions, num_samples=1)[-1, 0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
# Text generation requires a batch_size=1 model.
keras_model_batch1 = load_model(batch_size=1)
print(generate_text(keras_model_batch1, 'What of TensorFlow Federated, you ask? '))

What of TensorFlow Federated, you ask? The honour
of a daughter of Business, there was yet so aristocrat, an enjoying the
brill again.

The faces consisted.

"Is there no good that was a profounder room, they are still together; he w


## Load the Federated Shakespeare Data

The tff.simulation.datasets package provides a variety of datasets that are split into "clients", where each client corresponds to a dataset on a particular device that might participate in federated learning.

These datasets provide realistic non-IID data distributions that replicate in simulation the challenges of training on real decentralized data.

In [ ]:
train_data, test_data = tff.simulation.datasets.shakespeare.load_data()

1851392/1848122 [==============================] - 0s 0us/step


You may want to use your own dataset with TFF. To demonstrate loading a CSV file into TFF, let's write the data from the Shakespeare datasets to CSV and download the files.

In [ ]:
def write_data_to_csv_file(tff_dataset, f):
  f.write('"character","snippets"\n')
  for client_id in tff_dataset.client_ids:
    tf_dataset = tff_dataset.create_tf_dataset_for_client(client_id)
    for element in tf_dataset.as_numpy_iterator():
      # The CSV standard specifies that double quotes in the data must be
      # escaped by preceding them with another double quote.
      f.write('"' + client_id + '","' + str(element['snippets'], 'ascii')
        .replace('"', '""') + '"\n')

In [ ]:
filenames = ['shakespeare_train.csv', 'shakespeare_test.csv']
for filename, data in zip(filenames, [train_data, test_data]):
  with open(filename, 'w') as f:
    write_data_to_csv_file(data, f)

Let's see what the first few lines of each file look like.

In [ ]:
for filename in filenames:
  with open(filename, 'r') as f:
    print("~~~~~~~~~Reading file " + filename + "~~~~~~~~~")
    for i in range(10):
      print(f.readline())

~~~~~~~~~Reading file shakespeare_train.csv~~~~~~~~~
"character","snippets"

"ALL_S_WELL_THAT_ENDS_WELL_ADAM","Yonder comes my master, your brother."

"ALL_S_WELL_THAT_ENDS_WELL_ADAM","Come not within these doors; within this roof

The enemy of all your graces lives.

Your brother- no, no brother; yet the son-

Yet not the son; I will not call him son

Of him I was about to call his father-

Hath heard your praises; and this night he means

To burn the lodging where you use to lie,

And you within it. If he fail of that,

~~~~~~~~~Reading file shakespeare_test.csv~~~~~~~~~
"character","snippets"

"ALL_S_WELL_THAT_ENDS_WELL_ADAM","I down, and measure out my grave. Farewell, kind master.

So had you need;"

"ALL_S_WELL_THAT_ENDS_WELL_ADAM","your service. God be with my old master! He would not have spoke

such a word.

                                     Exeunt ORLANDO and ADAM

What, my young master? O my gentle master!"

"ALL_S_WELL_THAT_ENDS_WELL_AEDILE","I have."

"ALL_S_WELL_THAT_E

In [ ]:
from google.colab import files

for filename in filenames:
  files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Now we can upload the CSV files, as if we had the data in CSV format locally in the first place.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving shakespeare_test.csv to shakespeare_test (1).csv
Saving shakespeare_train.csv to shakespeare_train (1).csv
User uploaded file "shakespeare_test.csv" with length 441746 bytes
User uploaded file "shakespeare_train.csv" with length 3049377 bytes


We now write a function that returns a TFF dataset given the CSV file contents. We use Pandas to read in the CSV data. Then we implement `create_tf_dataset_for_client_fn` which takes a client ID and returns a TensorFlow dataset for that client. 

In [ ]:
import pandas as pd
import io

client_id_colname = 'character' # the column that represents client ID
data_colname = 'snippets'

def create_tff_dataset_for_csv_file(filename):
  # Create a Pandas dataframe from the CSV file.
  df = pd.read_csv(io.StringIO(uploaded[filename].decode("ascii")))
  # Collect unique character names.
  client_ids = df[client_id_colname].unique().tolist()

  # Define a function that takes client ID and returns a tf.data.Dataset for
  # that client. The tf.data.Dataset should contain a dictionary for each
  # line spoken by the character, where the single key in each dictionary is
  # "snippets" and the value is a string tensor containing the text of the line.
  def create_tf_dataset_for_client_fn(client_id):
    # Retrieve only the rows corresponding to this client.
    client_data = df[df[client_id_colname] == client_id]
    # Filter out any rows without a snippet of text spoken by the character.
    client_data = client_data[client_data[data_colname].notnull()]
    # Select the data columns, discarding the client id column.
    client_data = client_data[[data_colname]]
    # Convert to a dictionary in the format
    # [{column1 : value1, column2 : value2}]
    records = client_data.to_dict('records')

    # Define a generator that outputs a map for each row with column names as
    # keys and row contents as values. In this example there is only one column,
    # 'snippets', but this approach is shown to demonstrate how one might
    # load a CSV file with more columns.
    def dataset_gen():
      for row in records:
        yield row
    # Generate a dataset for the client, specifying the output type explicitly
    # as otherwise Tensorflow expects a tensor as the toplevel type.
    return tf.data.Dataset.from_generator(dataset_gen,
                                          output_types={data_colname: tf.string},
                                          output_shapes={data_colname: []}
    )

  # Your code to call tff.simulation.ClientData.from_clients_and_fn to return a
  # TFF dataset.

In [ ]:
test_filename = 'shakespeare_test.csv'
test_data = create_tff_dataset_for_csv_file(test_filename)
train_filename = 'shakespeare_train.csv'
train_data = create_tff_dataset_for_csv_file(train_filename)

The datasets we just created consist of a sequence of maps from the key 'snippet' to 
string `Tensors`, one for each line spoken by a particular character in a
Shakespeare play. The client keys consist of the name of the play joined with
the name of the character, so for example `MUCH_ADO_ABOUT_NOTHING_OTHELLO` corresponds to the lines for the character Othello in the play *Much Ado About Nothing*. Note that in a real federated learning scenario
clients are never identified or tracked by ids, but for simulation it is useful
to work with keyed datasets.

Here, for example, we can look at some data from King Lear:

In [ ]:
# Here the play is "The Tragedy of King Lear" and the character is "King".
raw_example_dataset = train_data.create_tf_dataset_for_client(
    'THE_TRAGEDY_OF_KING_LEAR_KING')
# To allow for future extensions, each entry x is a dictionary with a single key
# 'snippets' which contains the text.
for x in raw_example_dataset.take(2):
  print(x)

{'snippets': <tf.Tensor: shape=(), dtype=string, numpy=b"Live regist'red upon our brazen tombs,\nAnd then grace us in the disgrace of death;\nWhen, spite of cormorant devouring Time,\nTh' endeavour of this present breath may buy\nThat honour which shall bate his scythe's keen edge,\nAnd make us heirs of all eternity.\nTherefore, brave conquerors- for so you are\nThat war against your own affections\nAnd the huge army of the world's desires-\nOur late edict shall strongly stand in force:\nNavarre shall be the wonder of the world;\nOur court shall be a little Academe,\nStill and contemplative in living art.\nYou three, Berowne, Dumain, and Longaville,\nHave sworn for three years' term to live with me\nMy fellow-scholars, and to keep those statutes\nThat are recorded in this schedule here.\nYour oaths are pass'd; and now subscribe your names,\nThat his own hand may strike his honour down\nThat violates the smallest branch herein.\nIf you are arm'd to do as sworn to do,\nSubscribe to your 

If for any reason the steps above to load from CSV didn't work for you, don't worry- just reload the original datasets from the tff.simulation.datasets package.

In [ ]:
# Only run this cell if the step above did not work.
train_data, test_data = tff.simulation.datasets.shakespeare.load_data()

## Preprocess the Data

We now use `tf.data.Dataset` transformations to prepare this data for training the char RNN loaded above. This section doesn't introduce any new TFF concepts,
so if you would like to use the time to ask a question on the Dory this would be a good time to take a break.


In [ ]:
# Construct a lookup table to map string chars to indexes,
# using the vocab loaded above:
table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        keys=vocab, values=tf.constant(list(range(len(vocab))),
                                       dtype=tf.int64)),
    default_value=0)

# We can see that the table takes a tensor of characters and produces a tensor 
# of integer indices.
indices = table.lookup(tf.strings.bytes_split("Hello World"))
print(indices)

tf.Tensor([ 8 64  2  2 23 13 31 23 45  2  0], shape=(11,), dtype=int64)


In [ ]:
# Input pre-processing parameters
SEQ_LENGTH = 100
BATCH_SIZE = 8
BUFFER_SIZE = 10000  # For dataset shuffling

# Map a single dict containing a snippet to a tensor of int64 indices.
def to_ids(x):
  s = tf.reshape(x['snippets'], shape=[1])
  chars = tf.strings.bytes_split(s).values
  ids = table.lookup(chars)
  return ids

# Given a batch of sequences, split each sequence into the first 
# SEQ_LENGTH and last SEQ_LENGTH characters. The first SEQ_LENGTH chars
# will be the model input and the last SEQ_LENGTH chars will be the model
# target.
def split_input_target(chunk):
  input_text = tf.map_fn(lambda x: x[:-1], chunk)
  target_text = tf.map_fn(lambda x: x[1:], chunk)
  return (input_text, target_text)

def preprocess(dataset):
  return (
      # Map ASCII chars to int64 indexes using the vocab
      dataset.map(to_ids)
      # Split into individual chars
      .unbatch()
      # Form example sequences of SEQ_LENGTH +1
      .batch(SEQ_LENGTH + 1, drop_remainder=True)
      # Shuffle and form minibatches
      .shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
      # And finally split into (input, target) tuples,
      # each of length SEQ_LENGTH.
      .map(split_input_target))

Note that in the formation of the original sequences and in the formation of
batches above, we use `drop_remainder=True` for simplicity. This means that any
characters (clients) that don't have at least `(SEQ_LENGTH + 1) * BATCH_SIZE`
chars of text will have empty datasets. A typical approach to address this would
be to pad the batches with a special token, and then mask the loss to not take
the padding tokens into account.

This would complicate the example somewhat, so for this tutorial we only use full batches, as in the
[standard tutorial](https://www.tensorflow.org/tutorials/sequences/text_generation).
However, in the federated setting this issue is more significant, because many
users might have small datasets.

Now we can preprocess our `raw_example_dataset`, and check the types:

In [ ]:
example_dataset = preprocess(raw_example_dataset)
print(example_dataset.element_spec)

(TensorSpec(shape=(8, 100), dtype=tf.int64, name=None), TensorSpec(shape=(8, 100), dtype=tf.int64, name=None))


## Compile the model and test on the preprocessed data

We loaded an uncompiled keras model, but in order to run `keras_model.evaluate`, we need to compile it with a loss and metrics. We will also compile in an optimizer, which will be used as the on-device optimizer in Federated Learning.

The original tutorial didn't have char-level accuracy (the fraction
of predictions where the highest probability was put on the correct
next char). This is a useful metric, so we add it.
However, we need to define a new metric class for this because 
our predictions have rank 3 (a vector of logits for each of the 
`BATCH_SIZE * SEQ_LENGTH` predictions), and `SparseCategoricalAccuracy`
expects only rank 2 predictions.

In [ ]:
class FlattenedCategoricalAccuracy(tf.keras.metrics.SparseCategoricalAccuracy):

  def __init__(self, name='accuracy', dtype=tf.float32):
    super().__init__(name, dtype=dtype)

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_true = tf.reshape(y_true, [-1, 1])
    y_pred = tf.reshape(y_pred, [-1, len(vocab), 1])
    return super().update_state(y_true, y_pred, sample_weight)

Now we can compile a model, and evaluate it on our `example_dataset`.

In [ ]:
BATCH_SIZE = 8  # The training and eval batch size for the rest of this tutorial.
keras_model = load_model(batch_size=BATCH_SIZE)
keras_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[FlattenedCategoricalAccuracy()])

# Confirm that loss is much lower on Shakespeare than on random data
loss, accuracy = keras_model.evaluate(example_dataset.take(5), verbose=0)
print(
    'Evaluating on an example Shakespeare character: {a:3f}'.format(a=accuracy))

# As a sanity check, we can construct some completely random data, where we expect
# the accuracy to be essentially random:
random_guessed_accuracy = 1.0 / len(vocab)
print('Expected accuracy for random guessing: {a:.3f}'.format(
    a=random_guessed_accuracy))
num_steps = 10
random_indexes = np.random.randint(
    low=0, high=len(vocab), size=1 * BATCH_SIZE * (SEQ_LENGTH + 1) * num_steps)
data = collections.OrderedDict(
    snippets=tf.constant(
        ''.join(np.array(vocab)[random_indexes]), shape=[1, 1]))
random_dataset = preprocess(tf.data.Dataset.from_tensor_slices(data))
loss, accuracy = keras_model.evaluate(random_dataset, steps=num_steps, verbose=0)
print('Evaluating on completely random data: {a:.3f}'.format(a=accuracy))

Evaluating on an example Shakespeare character: 0.409250
Expected accuracy for random guessing: 0.012
Evaluating on completely random data: 0.013


## Fine-tune the model with Federated Learning

TFF serializes all TensorFlow computations so they can potentially be run in a
non-Python environment (even though at the moment, only a simulation runtime implemented in Python is available). Even though we are running in eager mode, (TF 2.0), currently TFF serializes TensorFlow computations by constructing the
necessary ops inside the context of a "`with tf.Graph.as_default()`" statement.
Thus, we need to provide a function that TFF can use to introduce our model into
a graph it controls. We do this as follows:

In [ ]:
# Clone the keras_model inside `create_tff_model()`, which TFF will
# call to produce a new copy of the model inside the graph that it will 
# serialize. Note: we want to construct all the necessary objects we'll need 
# _inside_ this method.
def create_tff_model():
  keras_model_clone = tf.keras.models.clone_model(keras_model)

  # Your code to create and return a tff.learning.Model using 
  # tff.learning.from_keras_model, using the same loss function and list of 
  # metrics we used to compile the Keras model.

Now we are ready to construct a Federated Averaging iterative process, which we will use to improve the model (for details on the Federated Averaging algorithm, see the paper [Communication-Efficient Learning of Deep Networks from Decentralized Data](https://arxiv.org/abs/1602.05629)).

In [ ]:
# This command builds all the TensorFlow graphs and serializes them: 
fed_avg = tff.learning.build_federated_averaging_process(
    model_fn=create_tff_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(lr=0.5))

The initial state of the model produced by `fed_avg.initialize()` is based
on the random initializers for the Keras model, not the weights that were loaded,
since `clone_model()` does not clone the weights. To start training
from a pre-trained model, we set the model weights in the server state
directly from the loaded model.

In [ ]:
def init_state():
  # The state of the FL server, containing the model and optimization state.
  state = fed_avg.initialize()

  # Your code here to update the initial state with the weights from the
  # existing model.

As an alternative to federated evaluation, we can mix federated training and centralized evaluation. First, we will create a dataset for centralized evaluation.

In [ ]:
import functools

def data(client, source=train_data):
  return preprocess(
      source.create_tf_dataset_for_client(client)).take(5)

clients = test_data.client_ids[:10]
# We concatenate the test datasets for evaluation with Keras.
test_dataset = functools.reduce(
    lambda d1, d2: d1.concatenate(d2),
    [data(client, test_data) for client in clients])

Now let's write a function to evaluate our model using Keras.

In [ ]:
def keras_evaluate(state, round_num):
  keras_model = load_model(batch_size=BATCH_SIZE)
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[FlattenedCategoricalAccuracy()])
  
  # Your code here to take global model weights and push them back into a Keras
  #  model to use its standard `.evaluate()` method.
  

  loss, accuracy = keras_model.evaluate(test_dataset, steps=2, verbose=0)
  print('\tEval: loss={l:.3f}, accuracy={a:.3f}'.format(l=loss, a=accuracy))

Federated learning can be useful in a variety of scenarios. In a cross-silo setting, there are a few clients which are highly available. In a cross-device setting, we are dealing with potentially a very large population of user devices, only a fraction of which may be available for training at a given point in time. This is the case, for example, when the client devices are mobile phones that participate in training only when plugged into a power source, off a metered network, and otherwise idle.

Of course, we are in a simulation environment, and all the data is locally available. Typically then, when running simulations, we would simply sample a random subset of the clients to be involved in each round of training, generally different in each round.

Try implementing sampling without replacement so that in each round, you train on 3 new clients.



In [ ]:
REPORT_GOAL = 3

def sample_clients_without_replacement(remaining_clients):
  # Your code here to return datasets for 3 randomly sampled clients.

Finally, let's define a simple training loop.

In [ ]:
NUM_ROUNDS = 10
remaining_clients = train_data.client_ids.copy()

state = init_state()

for round_num in range(NUM_ROUNDS):
  print('Round {r}'.format(r=round_num))
  keras_evaluate(state, round_num)
  state, metrics = fed_avg.next(
      state,
      sample_clients_without_replacement(remaining_clients))
  print('\tTrain: loss={l:.3f}, accuracy={a:.3f}'.format(
      l=metrics['train']['loss'], a=metrics['train']['accuracy']))

keras_evaluate(state, NUM_ROUNDS + 1)

Round 0
	Eval: loss=3.539, accuracy=0.367
Chose clients: ['THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_CONSTABLE'
 'THE_TAMING_OF_THE_SHREW_MERCHANT'
 'THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_PETO']

	Train: loss=4.325, accuracy=0.068
Round 1
	Eval: loss=4.304, accuracy=0.109
Chose clients: ['THE_TRAGEDY_OF_KING_LEAR_SHYLOCK'
 'THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_CHANCELLOR'
 'THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_MELUN']

	Train: loss=4.215, accuracy=0.144
Round 2
	Eval: loss=4.211, accuracy=0.195
Chose clients: ['PERICLES__PRINCE_OF_TYRE_SON' 'THE_TRAGEDY_OF_KING_LEAR_BARNARDINE'
 'THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_LUCY']

	Train: loss=4.132, accuracy=0.179
Round 3
	Eval: loss=4.056, accuracy=0.237
Chose clients: ['ALL_S_WELL_THAT_ENDS_WELL_AUDREY' 'ALL_S_WELL_THAT_ENDS_WELL_SICINIUS'
 'MUCH_ADO_ABOUT_NOTHING_SECOND_SENATOR']

	Train: loss=4.080, accuracy=0.170
Round 4
	Eval: loss=3.977, accuracy=0.249
Chose clients: ['ALL_S_WELL_THAT_ENDS_WELL_ADAM' 'ALL_S_WELL_THAT_ENDS_WEL

## Experimenting with optimizers

Using the tf.keras.optimizers API, you can experiment with many more client optimizers.

For example, we could try using SGD with momentum on the clients.

In [ ]:
fed_avg = tff.learning.build_federated_averaging_process(
    model_fn=create_tff_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(lr=0.5, momentum=0.9))

On NLP datasets (such as Shakespeare), you may want to try an adaptive optimizer, such as Adam.

In [ ]:
fed_avg = tff.learning.build_federated_averaging_process(
    model_fn=create_tff_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(lr=0.1))

As you can find out by studying the paper on the [Federated Averaging](https://arxiv.org/abs/1602.05629) algorithm, achieving convergence in a system with randomly sampled subsets of clients in each round can take a while, and it would be impractical to have to run hundreds of rounds in this interactive tutorial.

However, if you train longer on more Shakespeare data, you should see a difference in the style of the text generated with the updated model:

In [ ]:
# Set our newly trained weights back in the originally created model.
keras_model_batch1 = load_model(batch_size=1)
keras_model_batch1.set_weights([v.numpy() for v in keras_model.weights])
# Text generation requires batch_size=1
print(generate_text(keras_model_batch1, 'What of TensorFlow Federated, you ask? '))

What of TensorFlow Federated, you ask? Why so gracious
crawlich in Suddenly to break into a large and glaring dread-expression, had reached the Father rode
before him--knew him. "Where it is, at Tellson's; it was
prematurs" said the ngr
